In [ ]:
from core.data_sources import CLOBDataSource
from core.services.backend_api_client import BackendAPIClient
import sys
import os
import pandas as pd
import json
from dotenv import load_dotenv

import logging
logging.getLogger("asyncio").setLevel(logging.CRITICAL)

load_dotenv()

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)
print(os.getenv("TRADING_HOST", "localhost"))
backend_api = BackendAPIClient(host=os.getenv("TRADING_HOST", "localhost"))
clob = CLOBDataSource()

In [ ]:
await backend_api.list_databases()

In [ ]:
selected_dbs = ['bots/archived/hummingbot-test_1_orugol-2024.11.27_01.45/data/test_1_orugol-2024.sqlite']
dbs = await backend_api.read_databases(selected_dbs)
dbs

In [ ]:
selected_db = dbs[0].copy()
for table_name in selected_db["tables"].keys():
    selected_db["tables"][table_name] = pd.DataFrame(json.loads(selected_db["tables"][table_name]))
selected_db["tables"]

In [ ]:

executors = selected_db["tables"]["executors"]
controllers = selected_db["tables"]["controllers"]
executors.info()

In [ ]:
controller_ids = list(controllers["id"].unique())
controller_ids

In [ ]:
selected_controller = "8uCGoaQhJHirY895PSqZn1mmXkxxCBQr23RvXHYevNGV"
selected_executors = executors[executors["controller_id"] == selected_controller]
selected_executors

In [ ]:
selected_config = json.loads(controllers.loc[controllers["id"] == selected_controller, "config"][0])
selected_config

In [ ]:
candles = await clob.get_candles(connector_name=selected_config["connector_name"],
                                 trading_pair=selected_config["trading_pair"],
                                 interval="1s",
                                 start_time=selected_executors.timestamp.min() - 60 * 10,
                                 end_time=selected_executors.close_timestamp.max() + 60 * 10,
                                 from_trades=True)

In [ ]:
connector_name=selected_config["connector_name"]
trading_pair=selected_config["trading_pair"]
interval="1s"
start_time=selected_executors.timestamp.min() - 60 * 10
end_time=selected_executors.close_timestamp.max() + 60 * 10
from_trades=True

In [ ]:
candles["quote_asset_volume"] = candles["close"] * candles["volume"]
missing_cols = ['n_trades', 'taker_buy_base_volume', 'taker_buy_quote_volume']
for connector_name, trading_pair, interval in clob._candles_cache:
    clob._candles_cache[(connector_name, trading_pair, interval)] = candles.data
clob.dump_candles_cache(root_path)

In [ ]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=True)
clob.load_candles_cache(root_path)

config_instance = backtesting.get_controller_config_instance_from_dict(selected_config)

backtesting.load_candles_cache_by_connector_pair(connector_name=connector_name,
                                                 trading_pair=trading_pair,
                                                 root_path=root_path)

In [ ]:
candles = clob._candles_cache[(connector_name, trading_pair, interval)].copy()
candles

In [ ]:
backtesting_result = await backtesting.run_backtesting(config_instance, candles.timestamp.min(), candles.timestamp.max(), interval)
backtesting_result

In [ ]:
bt_executors_df = backtesting_result.executors_df.copy()
bt_executors_df

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.add_trace(go.Candlestick(x=candles["timestamp"],
                             open=candles["open"],
                             high=candles["high"],
                             low=candles["low"],
                             close=candles["close"]
                             ), row=1, col=1)

fig.show()

In [ ]:
fig.show()

In [ ]:
candles.to_csv(os.path.join(root_path, "data", "post_run", "candles.csv"))
executors.to_csv(os.path.join(root_path, "data", "post_run", "executors.csv"))
bt_executors_df.to_csv(os.path.join(root_path, "data", "post_run", "bt_executors_df.csv"))